In [7]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import csv
from dateutil import parser

pd.set_option("display.max_colwidth", None)

In [8]:
def crawl_by_keyword(keyword):
    df=pd.read_csv('tuoitre_'+keyword+'.csv')
    keyword_lookup = keyword
    csv_file = open('tuoitre_'+keyword+'_extracted.csv', 'w', newline='',encoding='utf-8')
    writer = csv.writer(csv_file)
    writer.writerow(['Title', 'URL','Publish Date', 'Mentioned Times','Word Count'])

    for index, row in df.iloc[:].iterrows():
        title = row['Title']
        url = row['URL']
        news_url = 'https://tuoitre.vn'+url
        response = requests.get(news_url)
        if response.status_code == 403:
            break
        write_flag = True

        soup=BeautifulSoup(response.content,'html.parser')
        try: 
            publish_date = soup.find('div',{'data-role': 'publishdate'}).get_text()
            publish_date=publish_date.replace('\n', ' ').replace('\t', ' ')
            cleaned_publish_date = publish_date.strip()
        except AttributeError:
            print('Could not get date of publication of: \n' + str(news_url))
            write_flag =False

        try:
            news_content = soup.find('div',class_='detail__main').get_text()
            nofm = news_content.lower().count(keyword_lookup.lower())
            nofw = len(news_content.lower().split())
        except  AttributeError:
            print('Cant target news content of: \n' + str(news_url))
            try:
                
                news_content = soup.find('div',class_='detail__cmain').get_text()
                nofm = news_content.lower().count(keyword_lookup.lower())
                nofw = len(news_content.lower().split())
                print('Target cmain instead successfully \n')
            except:
                print('Cant target news content cmain of: \n' + str(news_url) + '\n')
                write_flag = False

        try:
            # Parse the cleaned time string using dateutil.parser
            parsed_time = parser.parse(cleaned_publish_date)
            
            # Extract the date component
            date_only = parsed_time.date()
        except ValueError:
            print("Invalid time format. of: \n" + str(news_url))
        except UnboundLocalError:
            print("UnboundLocalError of: \n" + str(news_url))
        if write_flag == True:
            writer.writerow([title, url,date_only,nofm,nofw])
    csv_file.close()

In [9]:
keywords = [
    "chuyển đổi số"]
for text in keywords:
    crawl_by_keyword(text)

Cant target news content of: 
https://tuoitre.vn/muc-do-chuyen-doi-so-quoc-gia-viet-nam-hien-nay-ra-sao-20230922071704585.htm
Target cmain instead successfully 

Cant target news content of: 
https://tuoitre.vn/tin-tuc-sang-9-9-tang-le-phi-dang-ky-cap-bien-xe-may-o-to-20230908150629175.htm
Target cmain instead successfully 

Could not get date of publication of: 
https://tuoitre.vn/chuyen-doi-so-tai-sacombank-chu-trong-genz-nhung-phai-hai-hoa-moi-the-he-khach-hang-20230811060344667.htm
Cant target news content of: 
https://tuoitre.vn/de-xuat-bo-cong-an-cho-ngan-hang-khai-thac-du-lieu-quoc-gia-ve-dan-cu-20230616160558635.htm
Target cmain instead successfully 

Cant target news content of: 
https://tuoitre.vn/tin-tuc-sang-15-6-238-ca-covid-19-moi-26-nguoi-phai-tho-oxy-20230614092439783.htm
Target cmain instead successfully 

Could not get date of publication of: 
https://tuoitre.vn/ceo-ngan-hang-acb-chuyen-doi-so-phai-quyet-liet-va-di-vao-thuc-chat-20230606092043324.htm
Cant target news 